In [ ]:
from sklearn.cluster import KMeans
from sklearn import metrics
from scipy.spatial.distance import cdist
import numpy as np
import matplotlib.pyplot as plt
import openai
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from sklearn.cluster import KMeans
from IPython.display import display, Markdown, Latex
import pickle

In [ ]:
# # Function to generate embeddings for a given text
# def generate_embedding(text, model="text-embedding-ada-002"):
#     openai.api_key = api_key
#     return openai.Embedding.create(input = [text], model=model)['data'][0]['embedding']

# # Function to process a single row and generate embeddings for both columns
# def process_row(row):
#     print(row)
#     return generate_embedding(row)

# # Create a DataFrame (you can load your own data using pd.read_csv or another method)
# df = pd.read_csv('aita.csv')

# # Extract the results and add them as new columns to the DataFrame
# df["title_embedding"] = df["title"].apply(process_row).apply(literal_eval)

# print("Finished title embeddings.")

# print("Done")

df = pd.read_pickle("data/post_data.pkl")[["title", "title_embedding"]]
df.head()

In [ ]:
X = np.array(df["title_embedding"].to_list())

X

In [ ]:
# import pandas as pd
# from sklearn.cluster import KMeans

# distortions = []
# inertias = []

# K = range(1, 100)

# fig, ax = plt.subplots(3, 2, figsize=(15,8))

# for i in K:    
#     km = KMeans(n_clusters=i, init='k-means++', n_init=10, max_iter=100, random_state=42).fit(X)
    
#     inertias.append(km.inertia_)

inertias_df = pd.read_pickle('data/inertias.pkl')

inertias_df["1d"] = inertias_df["inertias"].diff()
inertias_df["2d"] = inertias_df["1d"].diff()
inertias_df["3d"] = inertias_df["2d"].diff()

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(10, 6))
axes[0, 0].plot(inertias_df.index, inertias_df["inertias"])
axes[0, 1].plot(inertias_df.index, inertias_df["1d"], color='orange')
axes[1, 0].plot(inertias_df.index, inertias_df["2d"], color='green')
axes[1, 1].plot(inertias_df.index, inertias_df["3d"], color='red')

for ax in axes.flat:
    ax.set_xlabel('Num clusters')
    ax.legend()

axes[0, 0].set_title('Raw inertias')
axes[0, 1].set_title('1d')
axes[1, 0].set_title('2d')
axes[1, 1].set_title('3d')

plt.tight_layout()
plt.show()

In [ ]:
km = KMeans(n_clusters=20, init='k-means++', n_init=10, max_iter=100, random_state=42)
km.fit(X)
df["cluster"] = km.labels_
df[["title", "cluster"]].head()

In [ ]:
distance = []

for i, distances in enumerate(km.transform(X)):
    sample_cluster = df["cluster"].iloc[i]
    distance_to_cluster = distances[sample_cluster]
    distance.append(distance_to_cluster)
    
df["distance"] = distance
df.head()

In [ ]:
# # Set your API key
# api_key = <key>
# openai.api_key = api_key

# reasons = []

# for k in range(0, 19):
#     posts = df.loc[df["cluster"] == k].sort_values(by="distance")["title"].values
        
#     question = f"I ran a clustering algorithm on embeddings of post titles on a subreddit called 'r/AmItheAsshole'. \
#         I'd like you to read through as many of these post titles as you can and sum up \
#         exactly ONE main reason why post submitters in this cluster are assholes. \
#         Please be as specific as possible, trying to avoid broad themes like selfishness and lack of empathy etc \
#         and instead opt for specific reasons like 'these people messed up weddings'. \
#         Avoid listing specific posts and commenting on the specific posts. \
#         As far as tone goes, please be as humorous and crass as possible. \
#         Here are the posts for cluster {k}: {posts[0:50]}"
    
#     try:
#         # Call the OpenAI API
#         response = openai.ChatCompletion.create(
#             model="gpt-3.5-turbo",
#             messages=[
#                 {"role": "user", "content": question},
#             ]
#         )
#     except Exception as e:
#         print(e)

#     # Extract and print the answer from the API response
#     answer = response.choices[0]["message"]["content"]
#     reasons.append(answer)
    
# with open('responses.pkl', 'wb') as f:
#         pickle.dump(reasons, f)
        
with open('data/responses.pkl', 'rb') as f:
        responses = pickle.load(f)
        
for i, reason in enumerate(responses):
    display(Markdown("\n---\n" + reason))